In [30]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
import pandas_ta as ta
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import seaborn as sns

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 150})


In [12]:
import stumpy

In [2]:
def load_candles(pair='BTC_USDT', timeframe='1d'):
    odf = pd.read_json(f'../../freq-user-data/data/binance/{pair}-{timeframe}.json').dropna()
    odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
    # odf['idate'] = odf.date.dt.strftime('%Y%m%d')
    odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
    # odf = odf.sort_index()
    odf = odf[['open', 'high', 'low', 'close']]
    return odf

In [4]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .1, .02
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')
    return ax

In [3]:
odf = load_candles(pair='BTC_USDT', timeframe='4h')
odf.tail()

,open,high,low,close
date,,,,
2023-06-01 16:00:00,26894.47,27175.00,26656.00,26862.68
2023-06-01 20:00:00,26862.68,26942.98,26713.34,26817.93
2023-06-02 00:00:00,26817.93,27017.00,26505.00,27000.00
2023-06-02 04:00:00,27000.01,27213.00,26949.40,27049.65
2023-06-02 08:00:00,27049.65,27183.07,27049.38,27130.75


In [20]:
m=10
mp = stumpy.stump(wdf.close[-200:], m).T[0]

np.pad(mp, ((0,m-1)), 'constant', constant_values=np.nan) 

array([1.0483998956483989, 0.52774990887717, 0.5959720254664452,
       0.6546866114066523, 0.7611015012256623, 0.9278911755448078,
       1.0834506268119413, 1.6529283908998298, 2.071104412028011,
       1.7738180407941213, 1.7527128084152674, 1.4268062972552484,
       1.5246960429321403, 1.9385401374695113, 1.9746956157111608,
       2.6684697910173156, 2.372643336775289, 1.9185580315644502,
       1.7697949982195256, 1.8711462141246624, 1.602755859478412,
       1.2956745146825326, 1.1391784055197913, 0.9474720765658479,
       1.324257674618567, 1.6014587382129788, 1.5911528767916223,
       1.7269462512964364, 1.8456918748088136, 2.0758230312530266,
       1.6514806077368493, 1.4640168316490767, 1.244629945252333,
       0.5959720254664452, 0.6546866114066523, 1.0549057769355572,
       1.3790884312131375, 1.935413699938081, 2.087096912716994,
       1.324257674618567, 1.8823184991191213, 1.7734142640148498,
       1.5725888485696442, 1.622764624218846, 1.0238703831508436,
      

In [38]:

from matplotlib import pyplot as plt

strategy = 'persistence'
param_folder = f'.'
strategy_params_json = f'{param_folder}/{strategy}.json'

dlen = len(odf)

wsize = 0
nwindows = 0

sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=8)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)

sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
# sl_lag_n = IntSlider(value=5,min=1,max=50,step=1,description='lag_n',continuous_update=False)
# tx_max_rat = FloatText(value=0.05,min=0.01,max=0.1,step=0.001,description='max_rat',continuous_update=False)
# sl_lag = IntSlider(value=6,min=1,max=6 * 7,step=1,description='Lag',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
# dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)
# wrng_1 = IntRangeSlider(value=[0, 0],min=0,max=10,step=1,description='Rng 1',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)
# wrng_2 = IntRangeSlider(value=[0, 0],min=0,max=10,step=1,description='Rng 2',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)
sl_m = IntSlider(value=10,min=3,max=24,step=1,description='m',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)
# per_2 = IntSlider(value=0,min=0,max=10,step=1,description='per 2',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)
per_len = IntSlider(value=6,min=1,max=20,step=1,description='per_len',continuous_update=False)
tak_n = IntSlider(value=6,min=1,max=20,step=1,description='tak_n',continuous_update=False)

def update_sl_w_range(*args):
    global wsize
    wsize = 2 ** (sl_n2.value )
    nwindows = dlen // wsize * 2
    sl_w.max = nwindows
    # wrng_1.max = wrng_2.max = wsize - 1
    per_len.max = wsize
    # per_1.max = per_2.max = wsize - 1
    # rng_2.max = sl_w.max
sl_n2.observe(update_sl_w_range, 'value')

strategy_params = {
    "candles": {"wdg": chk_candles},
    # "lag": {"wdg": sl_lag},
    "per_len": {"wdg": per_len},
    "m": {"wdg": sl_m},
}
all_params = {
    'w2log': {'wdg': sl_n2}, 'w': {'wdg': sl_w},
    **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in all_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
data_1, data_2 = None, None
def printer(
        w2log, w, candles,  per_len, m
):
    
    global wdf
    global data_1, data_2
    
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    print(f'N={len(wdf)}; Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')
    # print(f'lag_n={lag_n}; min_rat={min_rat}; max_rat={max_rat}\n')

    mp = stumpy.stump(wdf.close, m).T
    ctmp = np.pad(mp[0], ((m-1,0)), 'constant', constant_values=np.nan) 
    fig = plt.figure(layout='constrained', figsize=(15,5))
    fig1,fig2 = fig.subfigures(2,1, wspace=0.001, hspace=0, height_ratios=[10,1])

# axsLeft = subfigs[0].subplots(1, 2, sharey=True)

    (ax1,ax2) = fig1.subplots(2, 1, height_ratios=[2,1], sharex=True);


    if candles:
        plot_candles(wdf=wdf,ax=ax1);


    ax1.plot(wdf.index, wdf.close.values)
    # wdf.close.plot(ax=ax1);
    
    ax2.plot(wdf.index, ctmp)
    # (rax1,rax2) = fig2.subplots(1, 2, width_ratios=[1,1]);
    # ax1.plot()
    ax1.grid();ax2.grid();

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in all_params.items()});
x = display(ui, out);






Output()